# `InfiniteOpt.jl`: The Basics
Now that we are familiarized with Julia and `JuMP.jl`, let's talk about how `InfiniteOpt.jl` builds upon `JuMP.jl` to allow us to intuitively model infinite-dimensional optimization problems.

## Resources
Again, there is only so much that we can cover in a tutorial. Helpful resources include:
- The documentation: https://pulsipher.github.io/InfiniteOpt.jl/stable/.
- The paper: https://www.sciencedirect.com/science/article/pii/S0098135421003458?via%3Dihub 
- The open-source paper: https://arxiv.org/abs/2106.12689
- The `InfiniteOpt.jl` forum: https://github.com/pulsipher/InfiniteOpt.jl/discussions

## Optimal Control in `JuMP.jl`
To motivate `InfiniteOpt.jl`, let's consider modeling a simple optimal control problem in `JuMP.jl`.

### The Formulation
We'll take a look at a trajectory control problem:
$$
\begin{aligned}
&&\underset{x(t), v(t), u(t)}{\text{min}} &&& \int_{t \in \mathcal{D}_t} ||u(t)||_2^2 dt  \\
&&\text{s.t.} &&& v(0) = v0\\
&&&&& \frac{dx}{dt} = v(t), && t \in \mathcal{D}_t\\
&&&&& \frac{dv}{dt} = u(t), && t \in \mathcal{D}_t\\
&&&&& x(t_j) = xw_j, && j \in J
\end{aligned}
$$
Here position $x(t) \in \mathbb{R}^2$ and velocity $v(t) \in \mathbb{R}^2$ are state variables. The control variable $u(t) \in \mathbb{R}^2$ controls the acceleration (thrust) of the vehicle. Here we wish to plan a path that passes through all the waypoints $xw_i$ and minimizes the thrust used.

We cannot directly model this with `JuMP.jl` because the variables are functions of time $t$ and the formulation contains an integrals/derivatives. This is an infinite-dimensional problem.

### Discretize
We can transform it into a finite optimization `JuMP.jl` can handle by discretizing the variables over time and applying appropriate approximations to the integral/derivatives. 

We will transform the model via:
- let $\mathcal{D}_t = [0, T]$ and choose $n$ time steps such that $\Delta t = \frac{T}{n}$
- discretize each infinite variable (e.g., $\{x_t: t \in \{0, 1, \dots, n\}$)
- replace the integral with a sum over time
- approximate the derivatives via implicit Euler

With these steps, we obtain:
$$
\begin{aligned}
&&\underset{x_t, v_t, u_t}{\text{min}} &&& \sum_{t \in \{1, \dots, n\}} u_{1,t}^2 + u_{2,t}^2  \\
&&\text{s.t.} &&& x_{i,t+1} = x_{i,t} + \Delta t v_{i,t+1}, && i \in I, t \in \{0, 1, \dots, n-1\}\\
&&&&& v_{i,t+1} = v_{i,t} + \Delta t u_{i,t+1}, && i \in I, t \in \{0, 1, \dots, n-1\}\\
&&&&& x_{i,t} = xw_{i,t}, && i \in I, t \in Tw \\
&&&&& v_{i,0} = v0_i, && i \in I \\
\end{aligned}
$$

This is a discrete time model that we can formulate with `JuMP.jl`, let's try it out!

### Exercise: Optimal Control in `JuMP.jl`
**Problem**
- Implement the above model in `JuMP.jl`
- Add variables
- Add the objective
- Add the constraints

In [ ]:
using JuMP, HiGHS

# Set the parameters
n = 60
Δt = 60 / n 
I = 1:2
v0 = zeros(2)

# Set the waypoint data
p = [1 4 6 1; 1 3 0 1]
Tw = [0, 25, 50, 60]
xw = JuMP.Containers.DenseAxisArray(p, I, Tw)

# Define the model
model = Model(HiGHS.Optimizer)

# PUT CODE HERE


# Optimize the model
optimize!(model)

# Get the results
if has_values(model)
    x_opt = value.(x)
end


In [ ]:
using Plots

scatter(xw[1,:].data, xw[2,:].data, label = "Waypoints")
plot!(x_opt[1, :].data, x_opt[2, :].data, label = "Trajectory")
xlabel!("x_1")
ylabel!("x_2")

This works for this simple case, but we can start to think about some limitations:
- What if we want to implement a non-uniform grid?
- What about implementing (and switching between) derivative approximations (e.g., orthogonal collocation)?
- What about higher fidelity integral approximations?
- What about managing differing grid point between varied integral/derivative approximation schemes?
- What if we want to solve our model without discretization?
- What if we want to add uncertainty and/or PDE constraints?

We could resolve the majority of the above points by manually deriving a finite formulation for `JuMP.jl` each time, but this is cumbersome and prone to error. Hence, it is common to define the model once in discrete time and call it good.

However, characterizing models in discretized forms makes it likely for us to miss the insights we can gain from the infinite-dimensional form of the problem.

### Modeling in `InfiniteOpt.jl`
`InfiniteOpt.jl` is built upon our unifying abstraction for infinite-dimensional optimization. Moreover, it leverages `JuMP`'s architecture to have an intuitive interface.

Let's model the hovercraft optimal control problem in its native infinite-dimensional form using `InfiniteOpt.jl`:

In [ ]:
using InfiniteOpt, HiGHS

# Set the parameters
v0 = zeros(2)
I = 1:2
p = [1 4 6 1; 1 3 0 1]
Tw = [0, 25, 50, 60]
xw = JuMP.Containers.DenseAxisArray(p, I, Tw)

# Create the model
model = InfiniteModel(HiGHS.Optimizer)
@infinite_parameter(model, t ∈ [0, 60], num_supports = 61)
@variable(model, x[I], Infinite(t))
@variable(model, v[I], Infinite(t))
@variable(model, u[I], Infinite(t))
@objective(model, Min, ∫(u[1]^2 + u[2]^2, t))
@constraint(model, ∂.(x, t) .== v)
@constraint(model, ∂.(v, t) .== u)
@constraint(model, [i ∈ I], v[i](0) == v0[i])
@constraint(model, [i = I, j = Tw], x[i](j) == xw[i, j])

# Optimize the model
optimize!(model)

# Get the results
if has_values(model)
    x_opt = value.(x)
end


In [ ]:
using Plots

scatter(xw[1,:].data, xw[2,:].data, label = "Waypoints")
plot!(x_opt[1], x_opt[2], label = "Trajectory")
xlabel!("x_1")
ylabel!("x_2")

We were able to express the model in its infinite form and `InfiniteOpt.jl` took care of the rest!

Below we will walk through each step of what we just did and learn about how `InfiniteOpt.jl` works.

## Modeling in `InfiniteOpt.jl`
We will walk through each step of what we just did and learn about how `InfiniteOpt.jl` works.

### Infinite Models
